In [68]:
import pandas as pd
import numpy as np
import sys
import math
import joblib
from sklearn.metrics import mean_absolute_error,max_error,mean_squared_error,r2_score

In [69]:

# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
# df: es la matriz df.values [] , no incluye la etiqueta del grupo
def NormalizeViewMinable(df,valMin, dataRange):
    dataset_normalizado = np.empty((df.shape[0], df.shape[1]))
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]

    return dataset_normalizado





def predictionTest(dataset_test_norm, df_norm, vector_pesos_optmizacion, final_datset_join):
    df_groups_finally = df_norm.copy()
    #minDep = sys.float_info.max
    lista_grupos = []
    print("Longitud dataset norm: ",dataset_test_norm.shape[0])
    print("Vista minable : ",df_groups_finally.shape[0])
    #posMinDep = 0
    MAE = 0
    PSI15 =0 
    y_pred_test = []
    for z in range(int(dataset_test_norm.shape[0])):
        #print("Registro de Validación : ", z)
        #minDep = sys.float_info.max
        #posMinDep = 0
        #print("Minima Distancia: ", minDep)
        for k in range(int(df_groups_finally.shape[0])):
            ri = vector_pesos_optmizacion * np.power((dataset_test_norm[z]- df_groups_finally[k]),2) 
            dE = np.sum(ri)
            if dE < minDep:
                posMinDep = k
                minDep=dE
        
        #print("zz, " , z)
        # Grupo seleccionado
        print("Minima Distancia: ", minDep)
        print("Posicion: ", posMinDep)
        grupoSelected = int(final_datset_join.values[posMinDep][-1]) 
        lista_grupos.append(grupoSelected)
        print(grupoSelected)
    
    return lista_grupos

    

In [77]:
def predictionTesting(dataset_test_norm, df_norm, vector_pesos_optmizacion, final_datset_join, list_final_models,dataset_validation):
    df_groups_finally = df_norm.copy()
    print("Longitud dataset norm: ",dataset_test_norm.shape[0])
    print("Vista minable : ",df_groups_finally.shape[0])
    y_pred_test = []
    for z in range(int(dataset_test_norm.shape[0])):
        minDep = sys.float_info.max
        posMinDep = 0
        for k in range(int(df_groups_finally.shape[0])):
            ri = vector_pesos_optmizacion * np.power((dataset_test_norm[z]- df_groups_finally[k]),2) 
            dE = np.sum(ri)
            if dE < minDep:
                posMinDep = k
                minDep=dE
        
        #print("zz, " , z)
        # Grupo seleccionado
        grupoSelected = int(final_datset_join.values[posMinDep][-1])-1
        print(grupoSelected)
        #model_final, r2_final, yhat_final = LinearRegession(definitive_groups[i],0.1, 0.97).CalcularModeloLR() 
        psi_predicho = list_final_models[grupoSelected].predict(dataset_validation.values[z].reshape(1,-1))
        #print(psi_predicho)

        y_pred_test.append(psi_predicho[0])

    return y_pred_test



def metricasPerformanceCLR(y_true, yhat):

    R2 = r2_score(y_true, yhat)
    MAE =  mean_absolute_error(y_true, yhat)
    ME = max_error(y_true, yhat)
    MSE = mean_squared_error(y_true, yhat)
    print("Coeficiente de determinación : ", R2)
    print("Error absoluto medio: ", MAE)
    print("Maximo error residual: ", ME)
    print("RMSE: ", MSE)


In [35]:
# variables Generales
# 1. Conjunto de Test
dataset_test = pd.read_csv("DataOpt/dataset_test.csv")
# 2. Codificadores
valMin = np.loadtxt("DataOpt/Encoder_ValMin.txt")
dataRange = np.loadtxt("DataOpt/Encoder_dataRange.txt")


# 3. Vector de pesos mejor ajuste.
vp = np.loadtxt("DataOpt/bestFitnes.txt")

# 4. Vista minable de grupos
final_dataset_join = pd.read_csv("DataOpt/Final_dataset_join.csv")
# Normalización conjunto de test con los encoders
dataset_validation = dataset_test.copy()
dataset_validation = dataset_validation.drop(["ID_LOTE", "RDT_AJUSTADO"], axis=1)
dataset_validation.head(5)

# 5. cargue de los modelos
lista_modelos = []
for i in range(3):
    model = joblib.load(f'DataOpt/modelo_entrenado_{i}.pkl') # Carga del modelo.
    lista_modelos.append(model)


print(len(lista_modelos))



3


In [40]:
vector_pesos = vp.flatten()
print("Suma vector de pesos: ", np.sum(vector_pesos))

Suma vector de pesos:  1.0


In [41]:
vector_pesos

array([ 0.016,  0.012,  0.006,  0.004,  0.013,  0.013,  0.014,  0.008,
        0.   ,  0.011,  0.014,  0.014,  0.003,  0.001,  0.   ,  0.   ,
        0.   ,  0.006,  0.001,  0.011,  0.   ,  0.006,  0.   ,  0.014,
        0.012,  0.003,  0.015,  0.006,  0.004,  0.005,  0.005,  0.002,
        0.003,  0.014,  0.003,  0.002,  0.014,  0.002,  0.007,  0.007,
        0.014,  0.   ,  0.001,  0.006,  0.007,  0.002,  0.013,  0.006,
        0.006,  0.   ,  0.012,  0.007,  0.   ,  0.005,  0.011,  0.004,
        0.01 ,  0.002,  0.   ,  0.   ,  0.001,  0.   ,  0.004,  0.014,
        0.   ,  0.01 ,  0.016,  0.007,  0.006,  0.01 ,  0.007,  0.012,
        0.011,  0.013,  0.012,  0.002,  0.014,  0.008,  0.013,  0.003,
        0.   ,  0.   ,  0.013,  0.014,  0.   ,  0.009,  0.014,  0.   ,
        0.   ,  0.003,  0.003,  0.006,  0.014,  0.011,  0.003,  0.012,
        0.009,  0.004,  0.   ,  0.01 ,  0.006,  0.003,  0.001,  0.   ,
       -0.01 ,  0.004,  0.004,  0.002,  0.   ,  0.014,  0.006,  0.013,
      

In [5]:
#2.Normalizacion dataset | origina con agrupaciones
df_norm = NormalizeViewMinable(final_dataset_join.values[:,:-1],valMin, dataRange)
print(df_norm.shape)


# Normalización Conjunto de test
dataset_test_norm = NormalizeViewMinable(dataset_validation.values[:,:], valMin, dataRange)
print(dataset_test_norm.shape)



(759, 174)
(40, 174)


In [6]:
# Distribución de Grupos del dataset original.
final_dataset_join.Grupo.value_counts()

1    264
2    260
3    235
Name: Grupo, dtype: int64

In [18]:
grupos = predictionTest(dataset_test_norm, df_norm, vector_pesos, final_dataset_join)

Longitud dataset norm:  40
Vista minable :  759
Minima Distancia:  0.012795933488734477
Posicion:  467
2
Minima Distancia:  0.007452607414967675
Posicion:  719
3
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  0.004603958956642911
Posicion:  35
1
Minima Distancia:  0.03210673051463461
Posicion:  197
1
Minima Distancia:  0.010525092019897337
Posicion:  452
2
Minima Distancia:  0.021004111969365837
Posicion:  361
2
Minima Distancia:  0.013462158843972932
Posicion:  125
1
Minima Distancia:  -0.00017441793064843555
Posicion:  691
3
Minima Distancia:  0.033293331347076356
Posicion:  364
2
Minima Distancia:  0.034742162613345645
Posicion:  681
3
Minima Distancia:  0.02090356272805737
Posicion:  458
2
Minima Distancia:  0.031801823306771115
Posicion:  146
1
Minima Distancia:  0.002041041197707102
Posicion:  54
1
Minima Distancia:  -0.00877074351101669
Posicion:  757
3
Minima Distancia:  0.02127089413826398
Posicion:  439
2
Minima Distancia:  0.051601785164903144


In [25]:
lista_grupos = predictionTest(dataset_test_norm, df_norm, vector_pesos, final_dataset_join)

Longitud dataset norm:  40
Vista minable :  759
Minima Distancia:  0.012795933488734477
Posicion:  467
2
Minima Distancia:  0.007452607414967675
Posicion:  719
3
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  0.004603958956642911
Posicion:  35
1
Minima Distancia:  0.03210673051463461
Posicion:  197
1
Minima Distancia:  0.010525092019897337
Posicion:  452
2
Minima Distancia:  0.021004111969365837
Posicion:  361
2
Minima Distancia:  0.013462158843972932
Posicion:  125
1
Minima Distancia:  -0.00017441793064843555
Posicion:  691
3
Minima Distancia:  0.033293331347076356
Posicion:  364
2
Minima Distancia:  0.034742162613345645
Posicion:  681
3
Minima Distancia:  0.02090356272805737
Posicion:  458
2
Minima Distancia:  0.031801823306771115
Posicion:  146
1
Minima Distancia:  0.002041041197707102
Posicion:  54
1
Minima Distancia:  -0.00877074351101669
Posicion:  757
3
Minima Distancia:  0.02127089413826398
Posicion:  439
2
Minima Distancia:  0.051601785164903144


In [60]:
# Problema sin modificar la distancia
lista_grupos_f = predictionTest(dataset_test_norm, df_norm, vector_pesos, final_dataset_join)

Longitud dataset norm:  40
Vista minable :  759
Minima Distancia:  0.012795933488734477
Posicion:  467
2
Minima Distancia:  0.007452607414967675
Posicion:  719
3
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.0017547998563744492
Posicion:  412
2
Minima Distancia:  -0.00877074351101669
Posicion:  757
3
Minima Distancia:  -0.00877074351101669
Posicion:  757
3
Minima Distancia

In [78]:
yhat = predictionTesting(dataset_test_norm, df_norm, vector_pesos, final_dataset_join,lista_modelos, dataset_validation)

Longitud dataset norm:  40
Vista minable :  759
1
2
1
0
0
1
1
0
2
1
2
1
0
0
2
1
2
0
2
2
1
1
2
1
1
0
1
2
1
1
1
2
2
1
2
2
2
2
1
1


In [82]:
yhat

[4639.571004631529,
 7396.7184781930555,
 6141.0907001839005,
 4891.596111628416,
 4646.892624696979,
 6765.707495602663,
 5337.7705203465885,
 6271.741020961708,
 6088.46891234185,
 3917.2601886671255,
 1907.237941376101,
 2837.9894689287394,
 1077.5079794360718,
 4143.074187650971,
 4731.514655881656,
 6951.844858310615,
 2380.290505413497,
 6670.710416983864,
 4251.638366226231,
 6742.711822678,
 3826.7462278802195,
 4208.7362524532,
 3592.5807902751403,
 3210.8838120760993,
 7014.806701338268,
 4190.145341798063,
 2072.753085944256,
 5123.660103844915,
 2780.9792755791495,
 6236.773930820513,
 4751.0245843552475,
 5787.606555592312,
 3957.8007659915675,
 6175.8300123699955,
 5159.601921885784,
 3809.45935678386,
 3395.5871258614497,
 4185.1537008953455,
 6268.647592905174,
 5388.915417176024]

In [79]:
y_true = dataset_test.RDT_AJUSTADO

In [81]:
y_true

0     5983.72
1     6664.88
2     5302.33
3     5339.53
4     5720.93
5     6197.67
6     2002.33
7     6404.65
8     5488.37
9     4862.79
10    1871.63
11    3813.95
12    2072.09
13    4744.19
14    5651.16
15    5308.14
16    2604.65
17    5839.53
18    5469.77
19    7532.56
20    4000.00
21    4343.02
22    3627.91
23    3720.93
24    6325.58
25    4081.40
26    6613.95
27    3906.98
28    2637.21
29    5767.44
30    3813.95
31    5860.47
32    4651.16
33    4958.14
34    4988.37
35    4290.70
36    3377.91
37    4279.07
38    5339.53
39    5018.60
Name: RDT_AJUSTADO, dtype: float64

In [80]:
metricas = metricasPerformanceCLR(y_true, yhat)

Coeficiente de determinación :  0.26116722181191276
Error absoluto medio:  781.4805711645074
Maximo error residual:  4541.196914055744
RMSE:  1326945.026675017


In [76]:
yhat[0]

[4639.571004631529,
 7396.7184781930555,
 6141.0907001839005,
 4891.596111628416,
 4646.892624696979,
 6765.707495602663,
 5337.7705203465885,
 6271.741020961708,
 6088.46891234185,
 3917.2601886671255,
 1907.237941376101,
 2837.9894689287394,
 1077.5079794360718,
 4143.074187650971,
 4731.514655881656,
 6951.844858310615,
 2380.290505413497,
 6670.710416983864,
 4251.638366226231,
 6742.711822678,
 3826.7462278802195,
 4208.7362524532,
 3592.5807902751403,
 3210.8838120760993,
 7014.806701338268,
 4190.145341798063,
 2072.753085944256,
 5123.660103844915,
 2780.9792755791495,
 6236.773930820513,
 4751.0245843552475,
 5787.606555592312,
 3957.8007659915675,
 6175.8300123699955,
 5159.601921885784,
 3809.45935678386,
 3395.5871258614497,
 4185.1537008953455,
 6268.647592905174,
 5388.915417176024]